In [1]:
import os

In [2]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\Loan-Amount-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\Loan-Amount-Prediction'

In [5]:
# creating a data ingestion class
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path 

In [6]:
#import from constants and utils
from credit_risk.constants import *
from credit_risk.utils.common import read_yaml, create_directories


In [7]:
#creating a Configuration class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath   = CONFIG_FILE_PATH,
        params_filepath   = PARAMS_FILE_PATH,
        schema_filepath   = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)                       
        self.schema = read_yaml(schema_filepath)  
        
        create_directories([self.config.artifacts_root])  
        
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir     = config.root_dir,
            source_URL   = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir    = config.unzip_dir
        )
        return data_ingestion_config

In [8]:
#updating the components
import os
import urllib.request as request 
import zipfile
from credit_risk import logger
from credit_risk.utils.common import get_size

In [9]:
#creating data ingestion class
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file): 
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_file
                
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
            
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")    
    
    def extract_zip_file(self):
        """ArithmeticError
        zip_file_path:str
        Extracts the zip file to the specified directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)          

In [10]:
#creating the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config  = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-26 23:04:23,054: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-26 23:04:23,056: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-26 23:04:23,066: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-26 23:04:23,084: INFO: common: created directory at: artifacts]
[2025-02-26 23:04:23,093: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-26 23:04:25,001: INFO: 2466243107: artifacts/data_ingestion/credit_risk.zip download! with the following info: 
Connection: close
Content-Length: 372481
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "54a741a15377b7a90673028544a0e0b363107668d2f211acfc2e414e040e97f8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4CF0:257B06:CDED2:38B9D9:67BF81D4
Accept-Ranges: bytes
Da

The session ended at 51.47 mins